In [88]:
!pip install gymnasium


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [89]:
import os
import glob
import time

import torch
import torch.nn as nn
from torch.distributions import MultivariateNormal #dùng cho môi trường liên tục
from torch.distributions import Categorical #Lựa chọn hành động từ phân phối rời rạc.


import numpy as np

import gymnasium as gym

In [90]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


Tạo RolloutBuffer, lưu data lại để train, tạo hàm để update lúc train.

In [91]:
#RolloutBuffer Bộ nhớ tạm lưu giữ thông tin huấn luyện sau
#lưu trữ tạm thời các quỹ đạo 
#tập hợp dữ liệu về các tương tác của tác nhân (agent) với môi trường trong một tập hợp (episode), tính hàm lợi thế

class RolloutBuffer:
    def __init__(self):
        self.actions = [] #Lưu các hành động đã thực hiện
        self.states = []  #Lưu các trạng thái hiện tại
        self.logprobs = []#Lưu log của xác suất pi(a|s)
        self.rewards = [] #Lưu lại reward
        self.state_values = [] # Giá trị trạng thái V do critic dự đoán, để tính hàm lợi thế
        self.is_terminals = [] #Lưu lại cờ kết thúc


    def clear(self): #xóa buffer để update data mới
        del self.actions[:] 
        del self.states[:]
        del self.logprobs[:]
        del self.rewards[:]
        del self.state_values[:]
        del self.is_terminals[:]


#Actor: Mạng chính sách, xuất ra phân phối xác suất dựa trên trạng thái
#Critic: Mạng giá trị, Ước lượng giá trị trạng thái (Vt) để tính hàm ưu tiên
#hàm này giúp Agent chọn lọc hành động, đánh giá được trạng thái.
class ActorCritic(nn.Module):
    def __init__(self, state_dim, action_dim, has_continuous_action_space, action_std_init):
        #state_dim: Số chiều của state
        #action_dim: số chiều của action
        #xem action có liên tục không
        #action_std_init: độ lệch chuẩn cho phân phối liên tục
        super(ActorCritic, self).__init__()

        self.has_continuous_action_space = has_continuous_action_space

        #Nếu hành động liên tục lưu lại chiều hành động chuyển đến phương sai action_std_init^2

        if has_continuous_action_space:
            self.action_dim = action_dim
            self.action_var = torch.full((action_dim,), action_std_init * action_std_init).to(device)

        # ACTOR
        if has_continuous_action_space : # continuous action space
            self.actor = nn.Sequential(
                            nn.Linear(state_dim, 64),
                            nn.Tanh(),
                            nn.Linear(64, 64),
                            nn.Tanh(),
                            nn.Linear(64, action_dim),
                            nn.Tanh()
        #Nếu là hành động liên tục thì đầu ra là trung bình u(s) phân phối Gauss => phân phối hành động π(a|s) = N(u,tổng)
        #Tanh giới hạn đầu ra từ [-1,1], phù hợp vs phạm v hành động
                        )
        else: # discrete action space
            self.actor = nn.Sequential(
                            nn.Linear(state_dim, 64),
                            nn.Tanh(),
                            nn.Linear(64, 64),
                            nn.Tanh(),
                            nn.Linear(64, action_dim),
                            nn.Softmax(dim=-1)
                        )
        #Không gian rời rạc: xuất ra xác suất cho từng action rời rạc theo Softmax

        # CRITIC
        # Tương tự như actor nhưng xuất ra là 1 giá trị đơn ước lượng giá trị trạng thái Vt
        # Dùng để tính hàm lợi thế At = Q(s,a) - Vt
        self.critic = nn.Sequential(
                        nn.Linear(state_dim, 64),
                        nn.Tanh(),
                        nn.Linear(64, 64),
                        nn.Tanh(),
                        nn.Linear(64, 1)
                    )
        
    #Cập nhật phương sai cho hành động liên tục
    #tức là điều chỉnh khám phá, độ biến động cho các action
    def set_action_std(self, new_action_std):

        if self.has_continuous_action_space:
            self.action_var = torch.full((self.action_dim,), new_action_std * new_action_std).to(device)
        else:
            print("--------------------------------------------------------------------------------------------")
            print("WARNING : Calling ActorCritic::set_action_std() on discrete action space policy")
            print("--------------------------------------------------------------------------------------------")


    def forward(self):
        raise NotImplementedError


    
    def act(self, state): #đầu vào là state hiện tại

        # ACTOR
        if self.has_continuous_action_space:
            action_mean = self.actor(state) #Nếu đầu vào là state liên tục thì đầu ra actor là giá trị trung bình u phân phối gauss
            cov_mat = torch.diag(self.action_var).unsqueeze(dim=0) #phương sai Cov
            dist = MultivariateNormal(action_mean, cov_mat) #tạo phân phối đa biến
        else:
            action_probs = self.actor(state) #đầu ra softmax
            dist = Categorical(action_probs) #phân phối rời rạc

        action = dist.sample() #chọn hành động bằng cách lấy mẫu phân phối 
        action_logprob = dist.log_prob(action) #lấy log của các suất được chọn (π(a|s))

        # CRITIC
        state_val = self.critic(state) #xuất ra giá Vt từ critic

        return action.detach(), action_logprob.detach(), state_val.detach() #trả về action, log, Vt để lưu vào RolloutBuffer

    #Dùng để tính toán lại các giá trị cần thiết khi cập nhật danh sách:
        #+Log xác suất hành động 
        #+Entropy của chính sách
        #+Ước lượng giá trị trạng thái Vt
    # Đầu vào là state, action cũ đầu ra là logprob (mới), value (mới), entropy
    # So sánh policy cũ và mới.
    def evaluate(self, state, action):

        # ACTOR
        if self.has_continuous_action_space:
            action_mean = self.actor(state)
            action_var = self.action_var.expand_as(action_mean)
            cov_mat = torch.diag_embed(action_var).to(device)
            dist = MultivariateNormal(action_mean, cov_mat)

            # for single action continuous environments
            if self.action_dim == 1:
                action = action.reshape(-1, self.action_dim)

        else:
            action_probs = self.actor(state)
            dist = Categorical(action_probs)

        action_logprobs = dist.log_prob(action)
        dist_entropy = dist.entropy()

        # CRITIC
        state_values = self.critic(state)

        return action_logprobs, state_values, dist_entropy 
# Tại sao có act rồi cần dùng evaluate:
#    + act dùng để action và thu thập dữ liệu xong lưu vào RolloutBuffer để train
#    + evaluate dùng để tính toán train, policy cũ và mới: ratio, advantage, entropy.
# act → buffer → evaluate → loss → update.

Tạo 1 class Train model để có được policy

In [92]:
class PPO:
    def __init__(self, state_dim, action_dim, lr_actor, lr_critic, gamma, K_epochs, eps_clip, has_continuous_action_space, action_std_init=0.6):
    # state_dim, action_dim số chiều của trạng thái và hành động
    # lr_actor, lr_critic: tốc độ học cho actor và critic
    # gamma: hệ số chiết khấu phần thưởng trong tương lai.
    # K_epochs: số lần cập nhật chính sách cho mỗi batch.
    # eps_clip: hệ số clip PPO để giới hạn thay đổi chính sách.
    # action_std_init: độ lệch chuẩn ban đầu cho các hành động liên tục.
        self.has_continuous_action_space = has_continuous_action_space

        if has_continuous_action_space:
            self.action_std = action_std_init

        self.gamma = gamma
        self.eps_clip = eps_clip
        self.K_epochs = K_epochs

        self.buffer = RolloutBuffer() # để lưu các data trong 1 batch


        # Tạo 2 mạng:
        #    + policy: Mạng chính sách đang được huấn luyện 
        #    + policy_old: Mạng chính sách cũ, dùng để thu thập dữ liệu, ban đầu sao chép từ policy. 
        self.policy = ActorCritic(state_dim, action_dim, has_continuous_action_space, action_std_init).to(device)
        self.optimizer = torch.optim.Adam([
                        {'params': self.policy.actor.parameters(), 'lr': lr_actor},
                        {'params': self.policy.critic.parameters(), 'lr': lr_critic}
                    ])

        self.policy_old = ActorCritic(state_dim, action_dim, has_continuous_action_space, action_std_init).to(device)
        self.policy_old.load_state_dict(self.policy.state_dict())

        self.MseLoss = nn.MSELoss()

    # điều chỉnh độ lệch chuẩn để điều chỉnh độ khám phá 
    def set_action_std(self, new_action_std):

        if self.has_continuous_action_space:
            self.action_std = new_action_std
            self.policy.set_action_std(new_action_std)
            self.policy_old.set_action_std(new_action_std)

        else:
            print("--------------------------------------------------------------------------------------------")
            print("WARNING : Calling PPO::set_action_std() on discrete action space policy")
            print("--------------------------------------------------------------------------------------------")

    # giảm dần độ lệch chuẩn từ khám phá sang khai thác khi học đủ nhiều.
    def decay_action_std(self, action_std_decay_rate, min_action_std):
        print("--------------------------------------------------------------------------------------------")

        if self.has_continuous_action_space:
            self.action_std = self.action_std - action_std_decay_rate
            self.action_std = round(self.action_std, 4)
            if (self.action_std <= min_action_std):
                self.action_std = min_action_std
                print("setting actor output action_std to min_action_std : ", self.action_std)
            else:
                print("setting actor output action_std to : ", self.action_std)
            self.set_action_std(self.action_std)

        else:
            print("WARNING : Calling PPO::decay_action_std() on discrete action space policy")

        print("--------------------------------------------------------------------------------------------")

    #Chọn hành động và lưu lại thông tin
    def select_action(self, state):

        if self.has_continuous_action_space:
            with torch.no_grad():
                state = torch.FloatTensor(state).to(device) #chuyển state thành tensor
                action, action_logprob, state_val = self.policy_old.act(state)
                # Dùng policy_old để lấy action, action_logprob, state_val
    
            # Lưu lại vào RolloutBuffer
            self.buffer.states.append(state)
            self.buffer.actions.append(action)
            self.buffer.logprobs.append(action_logprob)
            self.buffer.state_values.append(state_val)

            return action.detach().cpu().numpy().flatten() #nếu không gian liên tục trả về action dưới dạng NumPy array.

        else:
            with torch.no_grad():
                state = torch.FloatTensor(state).to(device)
                action, action_logprob, state_val = self.policy_old.act(state)

            self.buffer.states.append(state)
            self.buffer.actions.append(action)
            self.buffer.logprobs.append(action_logprob)
            self.buffer.state_values.append(state_val)

            return action.item() #trả về action nếu không gian rời rạc

    #Cập nhật policy dựa trên dữ liệu trên bộ đệm, cải thiện chính sách qua K_epochs
    def update(self):

        # Monte Carlo estimate of returns
        rewards = [] #để lưu lại tổng reward mỗi lần khi train
        discounted_reward = 0 #để tính tống reward
        for reward, is_terminal in zip(reversed(self.buffer.rewards), reversed(self.buffer.is_terminals)): #lấy reward và is_terminals trong buffer
            #duyệt ngược lại dưới lên
            if is_terminal: 
                discounted_reward = 0 #nếu điểm cuỗi của 1 eposide thì phần thưởng về sau xóa
            discounted_reward = reward + (self.gamma * discounted_reward) #cộng phần thưởng hiện tại vs tương lai * gamma vì đi ngược dưới lên nên hợp lý
            rewards.insert(0, discounted_reward) # lưu tổng reward lại

        # Normalizing the rewards
        rewards = torch.tensor(rewards, dtype=torch.float32).to(device) #chuẩn hóa rewards thành tensor
        rewards = (rewards - rewards.mean()) / (rewards.std() + 1e-7) #đưa phần thưởng về 1 dạng chuẩn

        # convert list to tensor
        old_states = torch.squeeze(torch.stack(self.buffer.states, dim=0)).detach().to(device)
        old_actions = torch.squeeze(torch.stack(self.buffer.actions, dim=0)).detach().to(device)
        old_logprobs = torch.squeeze(torch.stack(self.buffer.logprobs, dim=0)).detach().to(device)
        old_state_values = torch.squeeze(torch.stack(self.buffer.state_values, dim=0)).detach().to(device)

        # calculate advantages
        advantages = rewards.detach() - old_state_values.detach() #tính toán hàm lợi thế


        # Optimize policy for K epochs
        for _ in range(self.K_epochs): 

            # Evaluating old actions and values
            logprobs, state_values, dist_entropy = self.policy.evaluate(old_states, old_actions) #để lấy hành động mới dưới chính sách hiện tại

            # match state_values tensor dimensions with rewards tensor
            state_values = torch.squeeze(state_values)

            # Finding the ratio (pi_theta / pi_theta__old)
            ratios = torch.exp(logprobs - old_logprobs.detach()) #ratios



            ## Xây dừng hàm loss clipped 
            # Finding Surrogate Loss
            surr1 = ratios * advantages
            surr2 = torch.clamp(ratios, 1-self.eps_clip, 1+self.eps_clip) * advantages

            # final loss of clipped objective PPO
            loss = -torch.min(surr1, surr2) + 0.5 * self.MseLoss(state_values, rewards) - 0.01 * dist_entropy #tổng loss

            # take gradient step xóa grad cũ của loss, tối ưu theo Adam
            self.optimizer.zero_grad()
            loss.mean().backward()
            self.optimizer.step()

        # Copy new weights into old policy
        self.policy_old.load_state_dict(self.policy.state_dict()) #chép từ policy sang policy_old

        # clear buffer
        self.buffer.clear()

    #lưu policy_old vào tệp 
    def save(self, checkpoint_path):
        torch.save(self.policy_old.state_dict(), checkpoint_path)

    #tải từ tệp model vào policy và policy_old
    def load(self, checkpoint_path):
        self.policy_old.load_state_dict(torch.load(checkpoint_path, map_location=lambda storage, loc: storage))
        self.policy.load_state_dict(torch.load(checkpoint_path, map_location=lambda storage, loc: storage))



## 1. Dùng select_action và policy_old để tương tác môi trường, lưu vào bộ đệm để train 
#  2. Sau khi buffer đầy thì dùng update để train policy
#  3. Sao chép policy sang policy_old
#  4. Xóa buffer, lặp lại từ 1.

Bắt đầu train

In [ ]:
################################### Training ###################################


####### initialize environment hyperparameters ######

env_name = "CartPole-v1"
has_continuous_action_space = False #không gian action rời rạc
best_reward = -float('inf')  # ban đầu là âm vô cực
max_ep_len = 1000                    # max timesteps in one episode
max_training_timesteps = int(3*1e5)   # break training loop if timeteps > max_training_timesteps -- dừng train khi số bước vượt quá

print_freq = max_ep_len * 4     # print avg reward in the interval (in num timesteps)
log_freq = max_ep_len * 2       # log avg reward in the interval (in num timesteps)


action_std = None

################ PPO hyperparameters ################

update_timestep = max_ep_len * 4      # update policy every n timesteps -- cập nhật policy
K_epochs = 40               # update policy for K epochs -- số lần lặp tối ưu trên 1 batch dữ liệu
eps_clip = 0.2              # clip parameter for PPO 
gamma = 0.99                # discount factor

lr_actor = 0.0003       # learning rate for actor network
lr_critic = 0.001       # learning rate for critic network


#####################################################


print("training environment name : " + env_name)

env = gym.make(env_name)

# state space dimension
state_dim = env.observation_space.shape[0]

# action space dimension
if has_continuous_action_space:
    action_dim = env.action_space.shape[0]
else:
    action_dim = env.action_space.n

# print("save checkpoint path : " + checkpoint_path)

#####################################################


############# print all hyperparameters #############

print("--------------------------------------------------------------------------------------------")

print("max training timesteps : ", max_training_timesteps)
print("max timesteps per episode : ", max_ep_len)
print("log frequency : " + str(log_freq) + " timesteps")
print("printing average reward over episodes in last : " + str(print_freq) + " timesteps")

print("--------------------------------------------------------------------------------------------")

print("state space dimension : ", state_dim)
print("action space dimension : ", action_dim)

print("--------------------------------------------------------------------------------------------")

if has_continuous_action_space:
    print("Initializing a continuous action space policy")
    print("--------------------------------------------------------------------------------------------")
    print("starting std of action distribution : ", action_std)
    print("decay rate of std of action distribution : ", action_std_decay_rate)
    print("minimum std of action distribution : ", min_action_std)
    print("decay frequency of std of action distribution : " + str(action_std_decay_freq) + " timesteps")

else:
    print("Initializing a discrete action space policy")

print("--------------------------------------------------------------------------------------------")

print("PPO update frequency : " + str(update_timestep) + " timesteps")
print("PPO K epochs : ", K_epochs)
print("PPO epsilon clip : ", eps_clip)
print("discount factor (gamma) : ", gamma)

print("--------------------------------------------------------------------------------------------")

print("optimizer learning rate actor : ", lr_actor)
print("optimizer learning rate critic : ", lr_critic)

#####################################################

print("============================================================================================")

################# training procedure ################

# initialize a PPO agent
ppo_agent = PPO(state_dim, action_dim, lr_actor, lr_critic, gamma, K_epochs, eps_clip, has_continuous_action_space, action_std)


# printing and logging variables
# print_running_reward = 0
# print_running_episodes = 0

# log_running_reward = 0
# log_running_episodes = 0

time_step = 0
i_episode = 0


# training loop
while time_step <= max_training_timesteps: #vòng lặp dừng khi vượt quá max_time_step

    state = env.reset() #trả về trạng thái ban đầu 

    # Check if state is tuple
    if isinstance(state, tuple):
            state = state[0]

    current_ep_reward = 0 #tổng phần thưởng trong episode hiện tại

    for t in range(1, max_ep_len+1): #chạy tối đa 500 bước

        # select action with policy
        action = ppo_agent.select_action(state) #chọn action cho state dự vào policy hiện tại
        state, reward, done, _ , _= env.step(action) #thực hiện hành động và nhận phản hồi

        # saving reward and is_terminals vào buffer
        ppo_agent.buffer.rewards.append(reward)
        ppo_agent.buffer.is_terminals.append(
            





            
        )

        time_step +=1
        current_ep_reward += reward

        # update PPO agent
        if time_step % update_timestep == 0:
            ppo_agent.update() #cập nhật lại policy nếu đủ số bước

        # if continuous action space; then decay action std of ouput action distribution
        # Nếu không gian hành động là liên tục, giảm độ lệch chuẩn để chuyển từ khám phá sang khai thác
        if has_continuous_action_space and time_step % action_std_decay_freq == 0:
            ppo_agent.decay_action_std(action_std_decay_rate, min_action_std)


        # break; if the episode is over
        if done:
            break


    print('Episode ', i_episode, ': Reward = ', current_ep_reward)

    # print_running_reward += current_ep_reward
    # print_running_episodes += 1

    # log_running_reward += current_ep_reward
    # log_running_episodes += 1
    # Save model if reward improves
    
    if current_ep_reward >= best_reward:
        print(f"Saving better model at episode {i_episode} with reward {current_ep_reward}")
        best_reward = current_ep_reward
        torch.save(ppo_agent.policy.state_dict(), 'ppo_best_model.pth')

    i_episode += 1

env.close()

training environment name : CartPole-v1
--------------------------------------------------------------------------------------------
max training timesteps :  300000
max timesteps per episode :  1000
log frequency : 2000 timesteps
printing average reward over episodes in last : 4000 timesteps
--------------------------------------------------------------------------------------------
state space dimension :  4
action space dimension :  2
--------------------------------------------------------------------------------------------
Initializing a discrete action space policy
--------------------------------------------------------------------------------------------
PPO update frequency : 4000 timesteps
PPO K epochs :  40
PPO epsilon clip :  0.2
discount factor (gamma) :  0.99
--------------------------------------------------------------------------------------------
optimizer learning rate actor :  0.0003
optimizer learning rate critic :  0.001
Episode  0 : Reward =  12.0
Saving better 

EVALUATION MODEL PPO 

In [105]:
import gymnasium as gym
import torch
import time

# ----- Load class ActorCritic và PPO của bạn trước đây -----
# from PPO import PPO

# ---------- Hyperparameters ----------
env_name = "CartPole-v1"
has_continuous_action_space = False

env = gym.make(env_name, render_mode="human")  # Dùng render_mode để xem animation
state_dim = env.observation_space.shape[0]
action_dim = env.action_space.n

# ---------- Khởi tạo PPO agent ----------
ppo_agent = PPO(state_dim, action_dim,
                lr_actor=0.0003,
                lr_critic=0.001,
                gamma=0.99,
                K_epochs=40,
                eps_clip=0.2,
                has_continuous_action_space=has_continuous_action_space)

# ---------- Load mô hình đã huấn luyện ----------
ppo_agent.policy.load_state_dict(torch.load('ppo_best_model.pth'))
ppo_agent.policy.eval()

# ---------- Hàm lấy hành động deterministic ----------
def get_deterministic_action(state):
    state = torch.FloatTensor(state).unsqueeze(0)
    with torch.no_grad():
        action_probs = ppo_agent.policy.actor(state)
        action = torch.argmax(action_probs, dim=1).item()
    return action

# ---------- Đánh giá mô hình ----------
num_eval_episodes = 1

for ep in range(num_eval_episodes):
    state, _ = env.reset()
    ep_reward = 0
    done = False

    step = 0
    while not done:
        action = get_deterministic_action(state)
        next_state, reward,done,_,_ = env.step(action)

        print(f"Step {step}: Action = {action}, Reward = {reward:.2f}")
        ep_reward += reward
        state = next_state
        step += 1
        if step >= 1000:
            break

        time.sleep(0.02)  # Để thấy rõ animation

    print(f"Episode {ep + 1}: Total Reward = {ep_reward:.2f}")

env.close()


Step 0: Action = 0, Reward = 1.00
Step 1: Action = 1, Reward = 1.00
Step 2: Action = 0, Reward = 1.00
Step 3: Action = 1, Reward = 1.00
Step 4: Action = 0, Reward = 1.00
Step 5: Action = 1, Reward = 1.00
Step 6: Action = 0, Reward = 1.00
Step 7: Action = 1, Reward = 1.00
Step 8: Action = 1, Reward = 1.00
Step 9: Action = 0, Reward = 1.00
Step 10: Action = 0, Reward = 1.00
Step 11: Action = 1, Reward = 1.00
Step 12: Action = 1, Reward = 1.00
Step 13: Action = 0, Reward = 1.00
Step 14: Action = 0, Reward = 1.00
Step 15: Action = 1, Reward = 1.00
Step 16: Action = 1, Reward = 1.00
Step 17: Action = 0, Reward = 1.00
Step 18: Action = 0, Reward = 1.00
Step 19: Action = 1, Reward = 1.00
Step 20: Action = 0, Reward = 1.00
Step 21: Action = 1, Reward = 1.00
Step 22: Action = 1, Reward = 1.00
Step 23: Action = 0, Reward = 1.00
Step 24: Action = 1, Reward = 1.00
Step 25: Action = 0, Reward = 1.00
Step 26: Action = 0, Reward = 1.00
Step 27: Action = 1, Reward = 1.00
Step 28: Action = 0, Reward = 